<a href="https://colab.research.google.com/github/brahimje/DataMining/blob/main/ImageMining/Atelier4_FeaturesEngineering_Brahim_JELLITE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <h1>Ingénierie des caractéristiques <br> (Features engineering)</h1></center>

Pour regarder la vidéo du support de cours:
https://youtu.be/OTEhvJhismM

A la fin, il faut remplir le formulaire ci-dessous utilisant les données de votre implémentation

https://forms.gle/1jmXku2ftAehbceRA

<h1> Plan </h1>

*	Introduction
*	Normalisation des caractéristiques
*	Sélection des caractéristiques
   -	Réduction
   -  Sélection
*	Implémentation


<h1> Introduction </h1>
Concevoir un modèle de classification performant repose sur plusieurs facteurs:

* Type et cohérence de la base données (base de données équilibrées)
* Choix des caractéristiques pertinentes
* Choix d’un algorithme de classification judicieux

D’après les experts dans le domaine:

* Les caractéristiques à utiliser en classification influencent plus que tout autre paramètre sur le résultat. 
* Aucun algorithme ne peut remplacer à lui seul le supplément que les caractéristiques procurent

Selon une enquête de Forbes (2016), les datascientists  consacrent 80 % (19% collection; 60% nettoyage et organisation) de leur temps à la préparation des données. 

<h1>Ingénierie de caractéristiques</h1>
L'Ingénierie de caractéristiques consiste en le traitement des caractéristiques dans le but d'améliorer le comportement du modéle de classification. <br>
Plusieurs étapes peuvent être utilisées (pas toutes utilisables pour l’image mining) 

* Valeurs manquantes (Non rencotrés en Image Mining)
* Valeurs aberrantes
* Transformation logarithmique
* Encodage unique (Concerne uniquement les étiquettes en Image Mining)
* Mise en échelle
* Sélection des caractéristiques


<h1> Implémentation </h1>

<h2> Préparation de l'environnement</h2>
Nous allons travailler dans cet atelier sur des caractéristiques déja extraites et enregistrées sur Google Drive  (Voir atelier tranfer Learning).

In [ ]:
#from keras.preprocessing import image
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h2> Chargement de la dataset</h2>
Pour tester et appliquer l'ingénierie de caractéristiques, nous allons utiliser des caractéristiques déjà extraites utilisant une architecture Deep Learning VGG16 de la base d'image Corel utilisée dans l'atelier de classification supervisée. 
Pour télécharger le fichier de caractéristiques et les étiquettes cliquer sur les liens ci-dessous :

[Features](https://drive.google.com/file/d/105zEqcYD1Deuzy5NM5dOPqnXDETUAfVt/view?usp=share_link)


[Labels](https://drive.google.com/file/d/1aggw9QXm9ao7CEmLa4ign9xi2zovUPnB/view?usp=drivesdk)

Vous pouvez utiliser le même chemin que celui que j'ai utilisé :
/content/drive/MyDrive/FeaturesEngineering/
- Créer un dossier sur la racine de votre drive "MyDrive" et nommé le: "FeaturesEngineering"
- Copier les fichiers "features_vgg16" et "labels" dans le dossier "FeaturesEngineering"

<h3>Charger les caractéristiques et les étiquettes.</h3>

In [ ]:
# Load features and labels
import pickle
fetauresPath='/content/drive/MyDrive/FeaturesEngineering/'
X = pickle.load( open( fetauresPath+"features_vgg16", "rb" ) )
y =pickle.load( open( fetauresPath+"labels", "rb" ) )

import numpy as np
print(np.array(X).shape, np.array(y).shape)

(490, 1000) (490,)


Pour chacune des étapes de l'ingénierie de caractéristiques, nous allons évaluer à chaque fois la classification afin de voir l'impact. 
Compléter l'implémentation de la fonction de classification. Utiliser un algorithme de classification de votre choix. 
La méthode doit afficher le taux de classification (accuracy) ainsi que le temps d'exécution de classification.

In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import time

def classify(X,y):
  # Compléter le code
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  clf = svm.SVC(kernel='linear', C=1, random_state=42)
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  
  accuracy = clf.score(X_test, y_test)
  print("Accuracy:", accuracy)
  scores = cross_val_score(clf, X, y, cv=5)

  # affichez les scores de chaque itération de la validation croisée
  print("Cross validation scores:", scores)


Classification utilisant les caractéristiques initiales sans manipulation

In [ ]:
# Afficher l'accuracy
classify(X,y)

Accuracy: 0.9489795918367347
Cross validation scores: [0.96938776 0.97959184 0.98979592 0.97959184 0.94897959]


<h1>Mise en échelle</h1>
Dans la plupart des cas, les caractéristiques de type numérique ne sont pas dans la même plage d’intervalle.
Exemple : les données sur les âges et les salaires différent et ne n’ont pas la même fourchette.
Pour un algorithme d’apprentissage automatique, les valeurs qui ne changent pas beaucoup même sur différentes échelles vont être très influentes en termes de classification.
Les algorithmes de classification basés sur le voisinage ou distance sont très sensible à la mise en échelle ; comme le KNN, K-means, …
Pour mettre en échelle les données nous avons deux solutions :

1.   Normalisation : min-max normalization
2.   Standardisation : z-score normalization



<h2> Standarization: z-score normalization</h2>
<center>z=(X-µ)/σ

X est l'ensemble d'apprentissage, µ est la moyenne et  σ est l’écart type
</center>
Le z-score met en échelle les valeurs en prenant en compte l’écart type (déviation standard). <br> 
le z-score réduit l’effet des données aberrantes

In [ ]:
"""Standarization
Standardize features by removing the mean and scaling to unit variance.
z = (x - u) / s
where u is the mean of the training samples or zero if with_mean=False
s is the standard deviation of the training samples or one if with_std=False.
"""
import numpy as np

def standardize(X):
    # Calculate the mean and standard deviation of the input data
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)

    # Standardize the data by subtracting the mean and dividing by the standard deviation
    z = (X - mean) / std
    return z

# Xzscore est la matrice de caractéristiques après standardisation
Xzscore = standardize(X)
#print(np.array(Xzscore).shape)
classify(Xzscore,y)

Accuracy: 0.9387755102040817
Cross validation scores: [0.98979592 0.96938776 0.98979592 0.94897959 0.93877551]


<h2>Normalization: min-max normalization</h2>
<center>Xnorm=(X-Xmin)/(Xmax -Xmin)</center>
Permet de mettre en échelle toutes les valeurs des caractéristiques dans une plage fixe entre 0 et 1.<br>
L'inconvénient de la normalization par min-max est qu'elle augmente les effets des valeurs aberrantes.<br>
Ainsi, avant la normalisation, il est recommandé de traiter les valeurs aberrantes.



In [ ]:
"""Noramlization
Xnorm=(X-Xmin)/(Xmax -Xmin).
"""
def normalize(X):
    # Calculate the minimum and maximum values of the input data
    Xmin = np.min(X, axis=0)
    Xmax = np.max(X, axis=0)

    # Normalize the data by subtracting the minimum value and dividing by the range
    Xnorm = (X - Xmin) / (Xmax - Xmin)

    return Xnorm

# Xminmax est la matrice de caractéristiques après normalisation
Xminmax = normalize(X)

#print(np.array(Xminmax).shape)
classify(Xminmax,y)

Accuracy: 0.9285714285714286
Cross validation scores: [0.96938776 0.96938776 0.96938776 0.97959184 0.95918367]


<h2> Transformation logarithmique</h2>
Pour éliminer les données aberrantes, une des techniques les plus utilisées est la transformation logarithmique.

*	Traiter des données biaisées et, après transformation, la distribution devient plus proche de la normale.
*	Réduire l’ordre de grandeur des données. Exemple : la différence de taille n’est pas de la même grandeur que la différence d'âges
*	Réduit aussi l'effet des valeurs aberrantes grâce à la normalisation des différences d'amplitude

NB : il ne faut pas que les données soient négatives





In [ ]:
# Transformation logarithmique
# Xlog est la matrice de caractéristiques après transformation logarithmique
Xlog= np.log(X)
# Classification utilisant les caractéristiques aprés tranformation logarithmique
classify(Xlog,y)

Accuracy: 0.9795918367346939
Cross validation scores: [1.         0.98979592 0.98979592 0.98979592 1.        ]


Par la suite nous allons procéder a la mise en échelle des données après transformation logarithmique

In [ ]:
# Normalization aprés TL
def normalize_after_log_transform(X):
    # Apply the logarithmic transformation to each element in the input data
    X_transformed = np.log(X)

    # Calculate the minimum and maximum values of the transformed data
    Xmin = np.min(X_transformed, axis=0)
    Xmax = np.max(X_transformed, axis=0)

    # Normalize the transformed data by subtracting the minimum value and dividing by the range
    Xnorm = (X_transformed - Xmin) / (Xmax - Xmin)

    return Xnorm

Xminmax_Log = normalize_after_log_transform(X)

classify(Xminmax_Log,y)

Accuracy: 0.9795918367346939
Cross validation scores: [1.         0.98979592 0.98979592 0.98979592 1.        ]


In [ ]:
# Standarization + TL
def standardize_and_log_transform(X):
    X_transformed = np.log(X)
    X_standardized = standardize(X)
    return X_standardized

Xzscore_Log = standardize_and_log_transform(X)
classify(Xzscore_Log,y)

Accuracy: 0.9387755102040817
Cross validation scores: [0.98979592 0.96938776 0.98979592 0.94897959 0.93877551]


<h1>Sélection de caractéristiques (features selection)</h1>
La sélection de caractéristiques (features selection) est l’opération permettant de d’extraire intelligemment de la connaissance à partir des données brutes. 

Objectif:
* Améliorer les performances (en termes de rapidité, de puissance de prédiction et de la simplicité du modèle).
* Réduire les dimensions et éliminer le bruit. 

La sélection des caractéristiques est un processus qui choisit un sous-ensemble optimal de caractéristique en fonction d'un critère donné.

Objectif de FS: <br>
* Supprimer les données non pertinentes,
* Augmenter de la précision du modèle d’apprentissage notamment en réduisant les besoins en stockage et les coûts de calcul,
* Réduire le nombre de données,
* Réduire la complexité de la description du modèle résultant,
* Améliorer la compréhension des données et du modèle.


<h2>Les catégories de la sélection des caractéristiques</h2>
Trois approches sont généralement utilisées dans la sélection des caractéristiques : embedded, wrapper et filter: <br>

* Les méthodes « embedded » intègrent directement la sélection dans le processus d’apprentissage. Les arbres de décision est une illustration de ce système. 
* Les méthodes « wrapper » optimisent explicitement un critère de précision, le plus souvent le taux d’erreur. Elles ne s’appuient en rien sur les caractéristiques de l’algorithme d’apprentissage qui est utilisé comme une boîte noire.
* les méthodes « filter » agissent en amont, avant la mise en œuvre de la technique d’apprentissage, et sans lien direct avec celui‐ci.


<h2> Suppression des caractéristiques à faible variance</h2>
Suppression des caractéristiques à faible variance (Removing features with low variance) est une méthode de sélection (élimination) basée sur le filtrage.<br>
On peut la considérer comme une méthode de nettoyage de caractéristique qui  élémine toutes les caractéristiques dont la variance n'atteint pas un certain seuil.<br>
Par défaut, elle supprime toutes les caractéristiques à variance nulle, c'est-à-dire les caractéristiques qui ont la même valeur dans tous les échantillons.

In [ ]:
# Nous allons utiliser les caractéristiques après normalisation Min-Max
# Xvth est la matrice de caractéristiques après suppression des caractéristiques (après normalisation Min-Max)  à faible variance
def remove_low_variance_features(X, threshold=0.1):
    
    Xnorm = normalize(X)

    # Calculate the variance of each feature
    variance = np.var(Xnorm, axis=0)

    # Select the features with variance above the threshold
    X_filtered = X[:, variance > threshold]

    return X_filtered

Xvth = remove_low_variance_features(X, threshold=0.001)
classify(Xvth,y)

Accuracy: 0.9489795918367347
Cross validation scores: [0.96938776 0.97959184 0.98979592 0.97959184 0.94897959]


<h2>chi2</h2>
C’est un algorithme de sélection des caractéristiques qui appartient à la famille des algorithmes de filtrage basé sur la statistique 𝜒2. Cette méthode mesure l’écart à l’indépendance entre une caractéristique et une classe. Elle commence par un niveau de signification élevé pour toutes les caractéristiques pour la discrétisation et chaque caractéristique est triée en fonction de ses valeurs.

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

def select_kbest_features(X, y, k=100):
    # Use the chi-squared test to select the k best features
    selector = SelectKBest(chi2, k=k)
    X_new = selector.fit_transform(X, y)

    return X_new

#Xchi2 est la matrice de caractéristiques après sélection de 100 caractéristiques utilisant chi2
Xchi2 = select_kbest_features(X, y, k=100)
classify(Xchi2,y)

Accuracy: 0.9489795918367347
Cross validation scores: [0.96938776 0.97959184 0.98979592 0.97959184 0.94897959]


<h2>Recursive Feature Elimination (RFE)</h2>
C’est une méthode de cartographie basée sur l'idée à plusieurs reprises, construire un modèle et choisir le meilleur ou le pire performant. Cette méthode qui appartient aux méthodes de filtrage est souvent utilisée comme étape de prétraitement pour les méthodes intégrée (souvent avec l’algorithme de classification SVM) afin de la généraliser à des grandes masses de données


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

model = SVC(kernel='linear')
rfe = RFE(model, n_features_to_select = 100)
rfe = rfe.fit(X, y)

# Xrfe est la matrice de caractéristiques après sélection de 100 caractéristiques utilisant RFE
Xrfel = rfe.transform(X)
classify(Xrfel,y)

Accuracy: 0.9489795918367347
Cross validation scores: [0.96938776 0.97959184 0.98979592 0.97959184 0.94897959]


Essayer d'identifier le nombre minimal de caractéristiques à utiliser pour obtenir le taux maximal de classification.

In [ ]:
# Code pour calculer le nombre minimal de caractéristiques à utiliser pour obtenir le taux maximal de classification
ranking = rfe.ranking_

max_score = 0
best_num_features = 0
for num_features in range(1, len(ranking) + 1):
    # Select the top-ranked features
    mask = ranking <= num_features
    X_selected = X[:, mask]

    # Evaluate the classification rate using cross-validation
    scores = cross_val_score(SVC(kernel='linear'), X_selected, y, cv=5)
    score = np.mean(scores)

    # Update the maximum score and the number of features that gave the maximum score
    if score > max_score:
        max_score = score
        best_num_features = num_features

print('The maximum classification rate is', max_score)
print('It is achieved with', best_num_features, 'features')

The maximum classification rate is 0.9734693877551021
It is achieved with 1 features


<h1>Relief</h1>
Tente de déterminer le plus proche voisin d'un certain nombre d'échantillons sélectionnés au hasard à partir de l'ensemble de données. Pour chaque échantillon sélectionné, les valeurs des caractéristiques sont comparées à ceux des voisins les plus proches et les scores pour chaque caractéristique sont mis à jour. L'idée est d'estimer la qualité des attributs en fonction de la qualité de leurs valeurs et faire la distinction entre des échantillons proches les uns des autres

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

selector = SelectKBest(mutual_info_classif, k=100)
X_new = selector.fit_transform(X, y)

# XRelief est la matrice de caractéristiques après sélection de 100 caractéristiques utilisant Relief
XRelief= X_new
classify(XRelief,y)


Accuracy: 0.9387755102040817
Cross validation scores: [0.96938776 0.97959184 0.98979592 0.97959184 0.94897959]


Essayer d'identifier le nombre minimal de caractéristiques à utiliser pour obtenir le taux maximal de classification.

In [ ]:
# Code pour calculer le nombre minimal de caractéristiques à utiliser pour obtenir le taux maximal de classification
ranking = selector.scores_
max_score = 0
best_num_features = 0
for num_features in range(1, len(ranking) + 1):
    # Select the top-ranked features
    mask = ranking <= num_features
    X_selected = X[:, mask]

    # Evaluate the classification rate using cross-validation
    scores = cross_val_score(SVC(kernel='linear'), X_selected, y, cv=5)
    score = np.mean(scores)

    # Update the maximum score and the number of features that gave the maximum score
    if score > max_score:
        max_score = score
        best_num_features = num_features

print('The maximum classification rate is', max_score)
print('It is achieved with', best_num_features, 'features')

The maximum classification rate is 0.9734693877551021
It is achieved with 1 features


<h1>Réduction de la dimensionnalité </h1>
La réduction de la dimensionnalité transforme les caractéristiques en une dimension inférieure. Elle peut être considérée comme étant une méthode de  Selection ou de création (extraction) de caractéristiques où nous dérivons des informations à partir de l’ensemble de caractéristiques de base pour construire un nouveau sous espace de caractéristiques. <br>
Ls approches de réduction de dimensionnalité les plus connues sont: PCA (Principal Component Analysis; Analyse en Composantes Principales (ACP)) et ICA (Independent Component Analysis, Analyse en Composantes Independantes (ACI)) 





<h2> ACP </h2>

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)

# Xpca est la matrice de caractéristiques après réduction de la dimensionalité utilisant l'ACP
# Essayer de changer la taille de réduction de l'ACP afin d'obtenir le meilleur taux de classification
Xpca=pca.fit_transform(X)
print(np.array(Xpca).shape)
classify(Xpca,y)

(490, 100)
Accuracy: 0.9489795918367347
Cross validation scores: [0.96938776 0.97959184 0.98979592 0.97959184 0.94897959]


<h2> ACI </h2>

In [ ]:
from sklearn.decomposition import FastICA
ica = FastICA(n_components=300)

# Xica est la matrice de caractéristiques après réduction de la dimensionalité utilisant l'ACI
# Essayer de changer la taille de réduction de l'ACI afin d'obtenir le meilleur taux de classification
Xica = ica.fit_transform(X)
print(np.array(Xica).shape)
classify(Xica,y)

(490, 300)
Accuracy: 0.7653061224489796
Cross validation scores: [0.85714286 0.82653061 0.82653061 0.83673469 0.83673469]
